In [1]:
import numpy as np
import pandas as pd
import os
from PIL import Image
from tqdm import tqdm 
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader, Subset, WeightedRandomSampler
from torchvision import transforms
from torchvision.models import resnet50, ResNet50_Weights
from collections import Counter
import random
from typing import Tuple, List

In [3]:
# --- A. 檔案路徑與常數設定 (指向您的工作目錄) ---
DATASET_ROOT = "."
TRAIN_IMAGES_DIR = os.path.join(DATASET_ROOT, "train", "images")
VAL_IMAGES_DIR = os.path.join(DATASET_ROOT, "val", "images")
WEIGHTS_SAVE_PATH = 'baseline_model.pth' 
RANDOM_STATE = 42

# ResNet50 標準參數
IMAGENET_MEAN = [0.485, 0.456, 0.406]
IMAGENET_STD = [0.229, 0.224, 0.225]
IMAGE_SIZE = 224
BATCH_SIZE = 32
LEARNING_RATE = 1e-5
EPOCHS = 50
PATIENCE = 5
DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(DEVICE)

torch.manual_seed(RANDOM_STATE)

# --- B. 建立全域標籤映射
def build_label_map(root_dir: str) -> Tuple[List[str], dict]:
    """掃描所有分區，建立 item_id 到 0~4599 數字標籤的映射"""
    all_item_ids = set()
    for split in ['train', 'val', 'test']:
        split_dir = os.path.join(root_dir, split, "images")
        if not os.path.exists(split_dir):
            continue
        for img_file in os.listdir(split_dir):
            if img_file.endswith('.jpg'):
                item_id = img_file.split("_")[0]
                all_item_ids.add(item_id)

    # 建立 item_id 字符串到整數標籤的映射
    sorted_ids = sorted(list(all_item_ids))
    item_to_idx = {item_id: idx for idx, item_id in enumerate(sorted_ids)}

    return sorted_ids, item_to_idx

ITEM_ID_LIST, ITEM_TO_IDX = build_label_map(DATASET_ROOT)
NUM_CLASSES = len(ITEM_ID_LIST)
print(f"從資料夾中重建標籤映射，共 {NUM_CLASSES} 個類別。")


# --- C. DeepFashionRetrievalDataset Class
class FolderBasedRetrievalDataset(Dataset):
    """直接從資料夾讀取檔案清單，並動態提取標籤。"""
    def __init__(self, images_dir: str, item_to_idx: dict, transform=None):
        self.images_dir = images_dir
        self.item_to_idx = item_to_idx
        self.transform = transform

        self.image_files = [f for f in os.listdir(images_dir) if f.endswith('.jpg')]
        
    def __len__(self):
        return len(self.image_files)

    def __getitem__(self, idx):
        img_name = self.image_files[idx]
        img_path = os.path.join(self.images_dir, img_name)
        
        # 1. 提取標籤
        item_id = img_name.split("_")[0]
        label = self.item_to_idx[item_id] # 取得 0~4599 的整數標籤

        # 2. 載入圖像
        image = Image.open(img_path).convert("RGB")
        
        # 3. 預處理
        if self.transform:
            image = self.transform(image)

        return image, label

# --- D. Transforms (ResNet50) 
# 確保 TRAIN 和 VAL 使用不同的 transforms
TRAIN_TRANSFORMS = transforms.Compose([
    transforms.Resize(256),              
    transforms.RandomResizedCrop(IMAGE_SIZE, scale=(0.8,1.0)), 
    transforms.RandomHorizontalFlip(),
    transforms.ColorJitter(brightness=0.2, contrast=0.2, saturation=0.2, hue=0.1),
    transforms.RandomRotation(15),
    transforms.RandomPerspective(distortion_scale=0.1, p=0.5),
    transforms.ToTensor(),
    transforms.Normalize(mean=IMAGENET_MEAN, std=IMAGENET_STD) 
])

VAL_TEST_TRANSFORMS = transforms.Compose([
    transforms.Resize(256),
    transforms.CenterCrop(IMAGE_SIZE), 
    transforms.ToTensor(),
    transforms.Normalize(mean=IMAGENET_MEAN, std=IMAGENET_STD)
])

cuda
從資料夾中重建標籤映射，共 4600 個類別。


In [4]:
# =======================================================
# 區塊 2：DataLoader 建立
# =======================================================

# 建立 Dataset
train_dataset = FolderBasedRetrievalDataset(TRAIN_IMAGES_DIR, ITEM_TO_IDX, TRAIN_TRANSFORMS)
val_dataset = FolderBasedRetrievalDataset(VAL_IMAGES_DIR, ITEM_TO_IDX, VAL_TEST_TRANSFORMS)

# --- E. 重新計算 WeightedRandomSampler (處理資料不平衡)
print("正在計算 WeightedRandomSampler 權重...")
train_labels = [train_dataset.item_to_idx[f.split("_")[0]] for f in train_dataset.image_files] 
class_sample_count = np.bincount(train_labels)
class_sample_count[class_sample_count==0] = 1 
weights = 1.0 / class_sample_count
sample_weights = np.array([weights[l] for l in train_labels])
sample_weights = torch.DoubleTensor(sample_weights)

sampler = WeightedRandomSampler(weights=sample_weights,
                                 num_samples=len(sample_weights),
                                 replacement=True)

# --- F. 建立 DataLoader ---
train_loader = DataLoader(train_dataset, batch_size=BATCH_SIZE, sampler=sampler, num_workers=4)
val_loader   = DataLoader(val_dataset, batch_size=BATCH_SIZE, shuffle=False, num_workers=4)

print(f"訓練集圖像數: {len(train_dataset)}。驗證集圖像數: {len(val_dataset)}")
print("✅ DataLoader 與 WeightedRandomSampler 已重新建立。")

正在計算 WeightedRandomSampler 權重...
訓練集圖像數: 13752。驗證集圖像數: 13752
✅ DataLoader 與 WeightedRandomSampler 已重新建立。


In [6]:
# =======================================================
# 區塊 3：Baseline 模型定義與訓練執行 (最終整合版)
# =======================================================

# --- G. 模型定義
def build_baseline_model(num_classes: int) -> nn.Module:
    model = resnet50(weights=ResNet50_Weights.DEFAULT) 
    num_ftrs = model.fc.in_features 
    model.fc = nn.Linear(num_ftrs, num_classes)
    return model

# --- H. 訓練和保存
def train_and_save_model(model: nn.Module, train_loader: DataLoader, val_loader: DataLoader, 
                         num_epochs: int, learning_rate: float, patience: int):
    
    model.to(DEVICE)
    criterion = nn.CrossEntropyLoss()
    optimizer = optim.Adam(model.parameters(), lr=learning_rate) 
    
    # Early Stopping 變數初始化
    best_val_loss = float('inf')
    epochs_no_improve = 0
    
    print(f"--- 開始訓練 (Epochs: {num_epochs}, Patience: {patience}) ---")
    
    for epoch in range(1, num_epochs + 1):
        # -------------------- 訓練步驟 --------------------
        model.train()
        running_loss, running_corrects = 0.0, 0
        for inputs, labels in tqdm(train_loader, desc=f"Epoch {epoch} Training"):
            inputs, labels = inputs.to(DEVICE), labels.to(DEVICE)
            optimizer.zero_grad()
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            
            # 準確率計算
            _, preds = torch.max(outputs, 1)
            running_corrects += torch.sum(preds == labels.data)
            
            loss.backward()
            optimizer.step()
            running_loss += loss.item()
        
        # 計算訓練結果
        avg_train_loss = running_loss / len(train_loader)
        avg_train_acc = running_corrects.double() / len(train_loader.dataset)

        
        # -------------------- 驗證步驟 --------------------
        model.eval()
        val_loss, val_corrects = 0.0, 0
        with torch.no_grad():
            for inputs, labels in tqdm(val_loader, desc=f"Epoch {epoch} Validation"):
                inputs, labels = inputs.to(DEVICE), labels.to(DEVICE)
                outputs = model(inputs)
                val_loss += criterion(outputs, labels).item()
                
                # 準確率計算
                _, preds = torch.max(outputs, 1)
                val_corrects += torch.sum(preds == labels.data)

        # 計算驗證結果
        avg_val_loss = val_loss / len(val_loader)
        avg_val_acc = val_corrects.double() / len(val_loader.dataset)

        
        # 輸出指標
        print(f"\n[結果] Epoch {epoch}, Train Loss: {avg_train_loss:.4f}, Train Acc: {avg_train_acc:.4f}, Val Loss: {avg_val_loss:.4f}, Val Acc: {avg_val_acc:.4f}")

        
        # -------------------- Early Stopping 和模型保存 --------------------
        if avg_val_loss < best_val_loss:
            best_val_loss = avg_val_loss
            epochs_no_improve = 0 # Val Loss 改善，重置計數器
            
            # 僅在 Val Loss 最佳時保存模型
            torch.save(model.state_dict(), WEIGHTS_SAVE_PATH)
            print(f"*** Val Loss 改善 ({best_val_loss:.4f})，模型權重已保存到 {WEIGHTS_SAVE_PATH} ***")
        else:
            epochs_no_improve += 1 # Val Loss 未改善，計數器 +1
            print(f"Val Loss 未改善 (連續 {epochs_no_improve} 個週期未改善).")

        # 檢查是否達到耐心上限
        if epochs_no_improve >= patience:
            print(f"\n🚨🚨 Early Stopping! Val Loss 已連續 {patience} 個週期未改善，停止訓練。🚨🚨")
            break # 跳出 Epoch 迴圈
            
    print("\n訓練流程結束。")


# --- I. 執行訓練 (修正呼叫)
if __name__ == '__main__':
    baseline_model = build_baseline_model(NUM_CLASSES)
    
    # 假設您已經定義了 NUM_EPOCHS, LEARNING_RATE, PATIENCE 常數
    train_and_save_model(baseline_model, train_loader, val_loader,
                         num_epochs=EPOCHS,
                         learning_rate=LEARNING_RATE,
                         patience=PATIENCE) # 傳入 PATIENCE 參數
    
    print("\n✅ 訓練完成。")

--- 開始訓練 (Epochs: 50, Patience: 5) ---


Epoch 1 Validation: 100%|██████████| 430/430 [00:44<00:00,  9.65it/s]



[結果] Epoch 1, Train Loss: 8.4232, Train Acc: 0.0007, Val Loss: 8.4097, Val Acc: 0.0007
*** Val Loss 改善 (8.4097)，模型權重已保存到 baseline_model.pth ***


Epoch 2 Validation: 100%|██████████| 430/430 [00:42<00:00, 10.01it/s]



[結果] Epoch 2, Train Loss: 8.3178, Train Acc: 0.0039, Val Loss: 8.2850, Val Acc: 0.0025
*** Val Loss 改善 (8.2850)，模型權重已保存到 baseline_model.pth ***


...


[結果] Epoch 48, Train Loss: 0.0756, Train Acc: 0.9929, Val Loss: 0.4815, Val Acc: 0.8989
*** Val Loss 改善 (0.4815)，模型權重已保存到 baseline_model.pth ***


Epoch 49 Validation: 100%|██████████| 430/430 [00:48<00:00,  8.87it/s]



[結果] Epoch 49, Train Loss: 0.0670, Train Acc: 0.9944, Val Loss: 0.4890, Val Acc: 0.8971
Val Loss 未改善 (連續 1 個週期未改善).


Epoch 50 Validation: 100%|██████████| 430/430 [00:42<00:00, 10.12it/s]


[結果] Epoch 50, Train Loss: 0.0681, Train Acc: 0.9921, Val Loss: 0.4908, Val Acc: 0.8941
Val Loss 未改善 (連續 2 個週期未改善).

訓練流程結束。

✅ 訓練完成。


In [7]:
# =======================================================
# 區塊 4：特徵提取器 (Extractor) 定義與測試
# =======================================================

# 確保所有參數與訓練時一致
EMBEDDING_DIM = 2048    # ResNet50 倒數第二層輸出維度
WEIGHTS_SAVE_PATH = 'baseline_model.pth' # 訓練產生的權重檔案名
DEVICE_EXTRACTOR = torch.device("cuda" if torch.cuda.is_available() else "cpu")


# ----------------------------------------------------
# A. 核心函數：載入特徵提取器
# ----------------------------------------------------
def load_embedding_extractor(weights_path: str = WEIGHTS_SAVE_PATH) -> nn.Module:
    """
    載入訓練好的權重，將模型轉換為特徵提取器 (移除分類層)。
    """
    if not os.path.exists(weights_path):
        raise FileNotFoundError(f"錯誤：找不到模型權重檔案 {weights_path}。請先完成訓練。")
        
    # 1. 建立與訓練時相同的完整模型結構
    # ⚠️ 這裡不需要下載 weights，因為我們將載入自己的權重
    model = resnet50(weights=None) 
    num_ftrs = model.fc.in_features
    # 替換為訓練時使用的分類層
    model.fc = nn.Linear(num_ftrs, NUM_CLASSES)

    # 2. 載入訓練好的權重
    model.load_state_dict(torch.load(weights_path, map_location='cpu'))
    
    # 3. 轉換為特徵提取器：將分類層替換為 nn.Identity() (即 2048-dim 特徵輸出)
    model.fc = nn.Identity()
    
    # 4. 設置為評估模式
    model.eval()
    return model.to(DEVICE_EXTRACTOR)

# ----------------------------------------------------
# B. 提取特徵函數
# ----------------------------------------------------
def extract_features(extractor: nn.Module, image_tensor: torch.Tensor, normalize: bool = True) -> torch.Tensor:
    """
    從圖像張量中提取特徵向量，並可選 L2 正規化。
    """
    image_tensor = image_tensor.to(DEVICE_EXTRACTOR)
    
    with torch.no_grad():
        embedding = extractor(image_tensor)
        
        if normalize:
            # 推薦對特徵進行 L2 正規化，以確保餘弦相似度計算的穩定性
            embedding = F.normalize(embedding, p=2, dim=1)
        
        return embedding.cpu().numpy() # 轉換為 numpy 陣列方便後續使用

# ----------------------------------------------------
# C. 模擬測試
# ----------------------------------------------------
try:
    # 1. 載入特徵提取器
    extractor = load_embedding_extractor()
    
    # 2. 從 val_loader 取一個 batch 進行測試
    # 這裡假設 val_loader 仍然可用，或者您可以手動載入一張圖片進行測試
    images, _ = next(iter(val_loader)) 
    
    # 3. 提取特徵
    feature_vector = extract_features(extractor, images)
    
    print("\n--- Extractor 測試成功 ---")
    print(f"提取出的特徵向量形狀 (Batch, Embedding_Dim): {feature_vector.shape}")
    print("✅ Baseline 特徵提取器已準備就緒！")

except Exception as e:
    print(f"\n⚠️ Extractor 測試失敗: {e}")
    print("請確保 baseline_model.pth 存在，且 val_loader 變數可用。")


--- Extractor 測試成功 ---
提取出的特徵向量形狀 (Batch, Embedding_Dim): (32, 2048)
✅ Baseline 特徵提取器已準備就緒！


In [8]:
# =======================================================
# 區塊 5：Recall@K 評估
# =======================================================
import torch.nn.functional as F
from sklearn.metrics.pairwise import cosine_similarity
import pandas as pd

# ----------------------------------------------------
# A. 提取所有特徵 (Gallery Features)
# ----------------------------------------------------
def extract_all_features(data_loader: DataLoader, extractor: nn.Module) -> Tuple[np.ndarray, np.ndarray]:
    """遍歷 DataLoader，提取所有圖像的特徵向量和對應標籤。"""
    
    all_features = []
    all_labels = []
    
    extractor.eval()
    
    print(f"開始提取 {len(data_loader.dataset)} 張圖像的特徵...")
    
    with torch.no_grad():
        for inputs, labels in tqdm(data_loader, desc="Extracting Features"):
            inputs = inputs.to(DEVICE)
            
            # 獲取 2048 維特徵
            features = extractor(inputs)
            
            # L2 正規化 (與 extract_features 函數一致)
            features = F.normalize(features, p=2, dim=1) 
            
            all_features.append(features.cpu().numpy())
            all_labels.append(labels.cpu().numpy())
            
    return np.concatenate(all_features), np.concatenate(all_labels)

# 載入提取器 (確保它已經被載入到記憶體中，如果沒有，請使用 load_embedding_extractor())
if 'extractor' not in locals():
    extractor = load_embedding_extractor() 


# 提取 Gallery (圖庫) 特徵：使用 val_loader 的資料
GALLERY_FEATURES, GALLERY_LABELS = extract_all_features(val_loader, extractor)

# 提取 Query (查詢) 特徵：通常使用一個獨立的查詢集。
# 由於 In-shop 檢索任務的特殊性，我們使用訓練集作為查詢集
QUERY_FEATURES, QUERY_LABELS = extract_all_features(train_loader, extractor)


# ----------------------------------------------------
# B. Recall@K 計算
# ----------------------------------------------------
def calculate_recall_at_k(query_features: np.ndarray, query_labels: np.ndarray, 
                          gallery_features: np.ndarray, gallery_labels: np.ndarray, 
                          ks: List[int]) -> dict:
    """計算 Recall@K 指標。"""
    
    # 計算所有 Query 對 Gallery 的餘弦相似度 (矩陣乘法，因為特徵已經 L2 正規化)
    print("\n計算相似度矩陣...")
    # shape: (Num_Queries, Num_Gallery)
    similarity_matrix = np.dot(query_features, gallery_features.T) 
    
    recalls = {k: 0 for k in ks}
    num_queries = len(query_labels)
    
    for i in tqdm(range(num_queries), desc="Calculating Recall@K"):
        q_label = query_labels[i]
        
        # 獲取相似度分數，並排序索引
        # 最高的相似度在最前面
        # ⚠️ 這裡使用 argsort 反向排序
        top_k_indices = np.argsort(similarity_matrix[i])[::-1]
        
        # 獲取排序後的 Gallery 標籤
        sorted_gallery_labels = gallery_labels[top_k_indices]
        
        # 過濾掉查詢圖像本身 (如果 Gallery 包含 Query)
        # 由於您使用了 train/val 兩個不同的 loader，這裡可以忽略這一複雜步驟
        
        # 檢查 Recall@K
        for k in ks:
            if k > len(sorted_gallery_labels): continue
            
            # 檢查 top-K 檢索結果中，是否有與 Query 相同標籤的圖像
            # (但不是它自己，如果 train/val 不分離則需要排除)
            if np.any(sorted_gallery_labels[:k] == q_label):
                recalls[k] += 1
                
    # 將計數轉換為比例
    recall_results = {f"Recall@{k}": count / num_queries for k, count in recalls.items()}
    return recall_results


# ----------------------------------------------------
# C. 執行評估
# ----------------------------------------------------
K_VALUES = [1, 5, 10, 20] # 業界常用 K 值

recall_metrics = calculate_recall_at_k(
    query_features=QUERY_FEATURES,
    query_labels=QUERY_LABELS,
    gallery_features=GALLERY_FEATURES,
    gallery_labels=GALLERY_LABELS,
    ks=K_VALUES
)

# 輸出結果
print("\n========== Baseline 模型 Recall@K 評估結果 ==========")
for k, score in recall_metrics.items():
    print(f"{k}: {score:.4f}")
print("======================================================")

print("✅ Baseline 檢索評估完成！")

開始提取 13752 張圖像的特徵...


Extracting Features: 100%|██████████| 430/430 [00:45<00:00,  9.39it/s]


開始提取 13752 張圖像的特徵...


Extracting Features: 100%|██████████| 430/430 [01:03<00:00,  6.82it/s]



計算相似度矩陣...


Calculating Recall@K: 100%|██████████| 13752/13752 [00:15<00:00, 881.39it/s]



========== Baseline 模型 Recall@K 評估結果 ==========
Recall@1: 0.9708
Recall@5: 0.9946
Recall@10: 0.9969
Recall@20: 0.9982
✅ Baseline 檢索評估完成！
